In [1]:
%%writefile '/public/home/lizw/task/pore_c/tools/porecscripts/v2.14/porecplot_pre_func.py'

#@Author: Zhuowen Li
#@LastEdit: 2021/9/30 下午1:37:37
#@Version: 
#@Description: small modification on color,etc.

#@Author: Zhuowen Li
#@LastEdit: 2021/9/22 下午3:10:26
#@Version: v2.9
#@Description: output singleton for formalization

#@Author: Zhuowen Li
#@LastEdit: 2021/9/10 下午1:13:53
#@Version: v2.8
#@Description: 
#1 update hic pre-processing file from pairsam files
#2 parquet instead of feather

#@Author: Zhuowen Li
#@LastEdit: 2021/8/4 下午2:32:46
#@Version: V2.3
#@Description: Prepare the tables for porecplot drawing, and basic statistic analysis of Pore-C data. 
#@Modifications: 1.fix the order distribtuion draw 2. split pair


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
from pathlib import PurePath
import seaborn as sns
import pypairix

from typing import (
    Tuple,
)

#font setting
import matplotlib.font_manager as font_manager
font_dirs = ["/public/home/lizw/software/font"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 20

def divide_order(x):
    if x <= 3:
        return str(x)
    elif (x == 4) or (x == 5):
        return '4-5'
    elif 6<=x<=10:
        return '6-10'
    elif 11<=x<=20:
        return '11-20'
    elif x > 20:
        return '20+'
             
def porec_merge(order:int,mpq:int,porec_dir:str,prefix:str,outdir:str) -> Tuple[pd.DataFrame,pd.DataFrame,pd.DataFrame]:
    align_dir = Path(PurePath(porec_dir,'align_table'))
    align_dir_porec_generator = align_dir.glob(prefix + "*pore_c.parquet")
    align_batch_porec_merge_df = pyarrow.concat_tables(list(map(pq.read_table,align_dir_porec_generator))).to_pandas()
    
    align_batch_porec_merge_df_pass = align_batch_porec_merge_df.query('pass_filter==True').query('mapping_quality>=@mpq')
    align_batch_porec_merge_df_singleton = align_batch_porec_merge_df.query("filter_reason=='singleton'").query('mapping_quality>=@mpq')

    align_allorder = pd.concat([align_batch_porec_merge_df_pass,align_batch_porec_merge_df_singleton],axis=0)
    
    #Importmant fixation in V2.1
    #According to PoreC : We refer to the set of filtered alignments associated with a given Pore-C read as a (multi-way) contact, and the number of fragments associated with a contact as its order.
    #When multiple alignment in the same read align to the same fragment, it would count multiple times when calculating order
    #Example fragmeng_id in read1 listed like this: 1,2,3,2,2,3,0   the order is 7 rather than just counting the fragment types. 
    align_allorder['order'] = align_allorder.groupby("read_name")['fragment_id'].transform('count')
    #use the mid point as final position: ref: ~/software/Pore-C-Snakemake_10Feb/.snakemake/conda/c2039cdc/lib/python3.8/site-packages/pore_c/analyses/contacts.py
    
    #modified according to porec_tools
    align_allorder = align_allorder.assign(
        pos = lambda x: np.rint((x.fragment_start + x.fragment_end)*0.5)
        .astype(int)
        )
    
    read_info =  align_allorder.reindex(columns=['read_name','read_length','order']).drop_duplicates()
    list_sorted = ['1','2','3','4-5','6-10','11-20','20+']
    read_info['divide_order'] = read_info['order'].apply(divide_order).astype('category').cat.set_categories(list_sorted) 
    #calculate the reads order distribution after drop duplicates
    order_count = read_info['order']
    read_info_outpath = PurePath(outdir,f'{prefix}.read_info.csv')
    read_info.to_csv(read_info_outpath)
    
    #pair_slice
    readID_order_map = read_info.groupby('divide_order')['read_name']
    pair_dir = Path(PurePath(porec_dir,'pairs'))
    pair_file = str(list(pair_dir.glob(prefix + "*.sorted.pairs.gz"))[0])
    pair_table = pd.read_table(
        pair_file,                     
        comment="#",             
        compression='gzip',
        names=['readID','chr1','pos1','chr2','pos2','strand1','strand2','pair_type','align1_idx','align2_idx','distance_on_read']
    )

    pair_table.replace({'readID': {'read|_[0-9]+': ''}},regex=True,inplace=True)
    pair_comment = pypairix.open(pair_file).get_header()

    for a,b in readID_order_map:
        pair_slice = pair_table.query('readID in @b')
        if len(pair_slice) != 0:
            pair_order_path = PurePath(outdir,f'{prefix}.order_{a}.pair')
            with open(pair_order_path,'w+') as pair_out:
                pair_out.write('\n'.join(pair_comment))
                pair_out.write('\n')
            pair_slice.to_csv(pair_order_path,header=None,index=None,mode='a',sep="\t")
                              
    #align_merge                                                                       
    align_merge_order_filter =  align_allorder.query('order>= @order')
    align_merge_order_filter_part = align_merge_order_filter.reindex(columns=['chrom','read_name','pos'])
    
    porec_merge_part_outpath = PurePath(outdir,f'{prefix}.order_{order}_merge_part.parquet')
    align_merge_order_filter_part.reset_index().to_parquet(porec_merge_part_outpath)
    
    #merge_keepall
    merge_keepall = align_allorder.query('order >=1')
    merge_keepall_part = merge_keepall.reindex(columns=['chrom','pos'])
    merge_keepall_part.sort_values(by=['chrom','pos'],inplace=True)
    
    singleton = align_allorder.query('order == 1')
    singleton_part = singleton.reindex(columns=['chrom','pos'])
    singleton_part.sort_values(by=['chrom','pos'],inplace=True)
    
    porec_keepall_outpath = PurePath(outdir,f'{prefix}.order_{order}_sorted_orderall_fornormailize.parquet')
    merge_keepall_part.reset_index().to_parquet(porec_keepall_outpath)
    
    singleton_outpah = PurePath(outdir,f'{prefix}.order_{order}_sorted_singleton_fornormailize.parquet')
    singleton_part.reset_index().to_parquet(singleton_outpah)
    
    return order_count,align_allorder,read_info

def pair_merge(pairfile,prefix,outdir):
    pairdf = pd.read_csv(pairfile,compression='gzip',sep="\t",comment="#",usecols=[0,1,2,3,4],names=['read_name','chrom1', 'pos1', 'chrom2', 'pos2'])
    pairdf1 = pairdf.reindex(['read_name','chrom1','pos1'],axis=1)
    pairdf2 = pairdf.reindex(['read_name','chrom2','pos2'],axis=1)
    pairdf1.columns=['read_name','chrom','pos']
    pairdf2.columns=['read_name','chrom','pos']
    pairdf_position = pd.concat([pairdf1,pairdf2])
    pairdf_position_sorted = pairdf_position.sort_values('read_name')

    hic_merge_part_outpath = PurePath(outdir,f'{prefix}.hic_merge_part.parquet')
    pairdf_position_sorted.reset_index().to_parquet(hic_merge_part_outpath)

    #merge_keepall
    merge_keepall_part = pairdf_position_sorted.reindex(columns=['chrom','pos'])
    merge_keepall_part.sort_values(by=['chrom','pos'],inplace=True)
    hic_keepall_outpath = PurePath(outdir,f'{prefix}.hic_sorted_orderall_fornormailize.parquet')
    merge_keepall_part.reset_index().to_parquet(hic_keepall_outpath)

def orderdistri(order_count,prefix,outdir):
    f, ax = plt.subplots(figsize=(8, 8))
    ax.set_title('Contact order distribution')
    ax.set_xlabel("Contact order")
    ax.set_ylabel("Frequency Density")
    ax.set_xlim(0,30)
    bins_count = order_count.max()
    order_count.hist(bins=np.arange(bins_count+1)-0.5,ax=ax,density=1,edgecolor = 'w',color='#4CB391')
    orderdistri_outpath = PurePath(outdir,f'{prefix}_Contact_order_distribution.png')
    plt.savefig(orderdistri_outpath,format = 'png',dpi=300,bbox_inches = 'tight')

def lengthdistri(align_allorder,prefix,outdir):
    f, ax = plt.subplots(figsize=(8, 8))
    ax.set_title('Read length distribution')
    ax.set_xlabel("Read length")
    ax.set_ylabel("Frequency Density")
    ax.set_xlim(0,15000)
    bins_count = align_allorder['read_length'].max()//500
    align_allorder['read_length'].hist(bins=bins_count+1,ax=ax,density=1,edgecolor = 'w',color='#4CB391')
    lengthdistri_outpath = PurePath(outdir,f'{prefix}_Read_length_distribution.png')
    plt.savefig(lengthdistri_outpath,format = 'png',dpi=300,bbox_inches = 'tight')

def lenorderbox(read_info,prefix,outdir):
    f, ax = plt.subplots(figsize=(25,8))
    read_info.boxplot(column='read_length',by=['order'],ax=ax)
    ax.set_ylim(0,30000)
    ax.set_ylabel("Length")
    ax.set_xlabel("Order")
    plt.title('')
    plt.suptitle('') 
    lenorderbox_outpath = PurePath(outdir,f'{prefix}_Read_length_order_distribution.png')
    plt.savefig(lenorderbox_outpath,format = 'png',dpi=300,bbox_inches = 'tight')

Writing /public/home/lizw/task/pore_c/tools/porecscripts/v2.14/porecplot_pre_func.py


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
from pathlib import PurePath
import seaborn as sns
porec_dir = '/public/home/lizw/task/pore_c/porec_1000_filter_mainchr_result'
prefix = 'DpnII_run04'
mpq=1
order=2
align_dir = Path(PurePath(porec_dir,'align_table'))
align_dir_porec_generator = align_dir.glob(prefix + "*pore_c.parquet")
align_batch_porec_merge_df = pyarrow.concat_tables(list(map(pq.read_table,align_dir_porec_generator))).to_pandas()

In [3]:
def divide_order(x):
    if x <= 3:
        return str(x)
    elif (x == 4) or (x == 5):
        return '4-5'
    elif 6<=x<=10:
        return '6-10'
    elif 11<=x<=20:
        return '11-20'
    elif x > 20:
        return '20+'

In [5]:
align_batch_porec_merge_df_pass = align_batch_porec_merge_df.query('pass_filter==True').query('mapping_quality>=@mpq')
align_batch_porec_merge_df_singleton = align_batch_porec_merge_df.query("filter_reason=='singleton'").query('mapping_quality>=@mpq')
align_allorder = pd.concat([align_batch_porec_merge_df_pass,align_batch_porec_merge_df_singleton],axis=0)
outdir = '/public/home/lizw/task/pore_c/tools/jp_note/new'

#Importmant fixation in V2.1
#According to PoreC : We refer to the set of filtered alignments associated with a given Pore-C read as a (multi-way) contact, and the number of fragments associated with a contact as its order.
#When multiple alignment in the same read align to the same fragment, it would count multiple times when calculating order
#Example fragmeng_id in read1 listed like this: 1,2,3,2,2,3,0   the order is 7 rather than just counting the fragment types. 
align_allorder['order'] = align_allorder.groupby("read_name")['fragment_id'].transform('count')
#use the mid point as final position: ref: ~/software/Pore-C-Snakemake_10Feb/.snakemake/conda/c2039cdc/lib/python3.8/site-packages/pore_c/analyses/contacts.py

align_allorder = align_allorder.assign(
    fragment_mid = lambda x: np.rint((x.fragment_start + x.fragment_end)*0.5)
    .astype(int)
    )
order_count = align_allorder['order']
#align_allorder['fragment_mid'] =  align_allorder.eval('fragment_end+fragment_start+1')//2
#modified according to porec_tools

read_info =  align_allorder.reindex(columns=['read_name','read_length','order']).drop_duplicates()
list_sorted = ['1','2','3','4-5','6-10','11-20','>20']
read_info['divide_order'] = read_info['order'].apply(divide_order).astype('category').cat.set_categories(list_sorted) 


In [ ]:
readID_order_map = read_info.groupby('divide_order')['read_name']
pair_dir = Path(PurePath(porec_dir,'pairs'))
pair_file = str(list(pair_dir.glob(prefix + "*.sorted.pairs.gz"))[0])

pair_table = pd.read_table(
    pair_file,                     
    comment="#",             
    compression='gzip',
    names=['readID','chr1','pos1','chr2','pos2','strand1','strand2','pair_type','align1_idx','align2_idx','distance_on_read']
)

pair_table.replace({'readID': {'read|_[0-9]+': ''}},regex=True,inplace=True)
import pypairix
pair_comment = pypairix.open(pair_file).get_header()

for a,b in readID_order_map:
    pair_slice = pair_table.query('readID in @b')
    pair_order_path = PurePath(outdir,f'{prefix}.order_{a}.pair')
    with open(pair_order_path,'w+') as pair_out:
        pair_out.write('\n'.join(pair_comment))
    pair_slice.to_csv(pair_order_path,header=None,index=None,mode='a',sep="\t")